In [1]:

import torch
import torch.nn as nn
from torch.nn import modules, functional as F
from DataGenerator import get_distance_matrices

class DMLoss(nn.Module):
    def __init__(self):
        super(DMLoss, self).__init__()
        self.reduction = 'sum'
    def forward(self, x, target):
        return F.mse_loss(get_distance_matrices(x), target,
         reduction=self.reduction)
# %%

class AutoEncoder(nn.Module):
    def __init__(self, in_dim: int, out_dim: int):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=2, kernel_size=2),
            nn.Conv2d(in_channels=2, out_channels=2, kernel_size=2),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(in_channels=2, out_channels=1, kernel_size=3),
            nn.Linear(in_features=in_dim, out_features=out_dim, bias=True)
        )
    def forward(self, x: torch.Tensor):
        encoded = self.encoder(x)
        # print("encoded: ", encoded.size())
        decoded = self.decoder(encoded)
        # print("decoded: ", decoded.size())
        b, _, n, m = decoded.size()
        return decoded.view((b, n, m))
# %%


In [2]:
from DataGenerator import DataGenerator

EPOCH = 100
dg = DataGenerator((5, 2))
dgl = dg.get_dm_loader(N=3, batch=10000)

ae = AutoEncoder(5, 2)
optim = torch.optim.Adam(ae.parameters(), lr=1e-5)

dml = DMLoss()

for epoch in range(EPOCH):
    for step, target in enumerate(dgl):        
        rs = ae(target)
        loss = dml(rs, target)
        optim.zero_grad()
        loss.backward()
        optim.step()
        print("Epoch: ", epoch, "| train loss: %.4f" %loss.data.numpy())
        break

Epoch:0| train loss: 20.3126


RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.